In [1]:
import mxnet as mx
from mxnet import gluon
from mxnet import autograd as autograd
from mxnet import nd

import utils


ctx = utils.try_gpu()
net = gluon.nn.Sequential()
with net.name_scope():
    net.add(gluon.nn.Conv2D(channels=3, kernel_size=5, activation='relu'))
    net.add(gluon.nn.MaxPool2D(pool_size=2, strides=2))
    net.add(gluon.nn.Conv2D(channels=50, kernel_size=3, activation='relu'))
    net.add(gluon.nn.MaxPool2D(pool_size=2, strides=2))
    net.add(gluon.nn.Flatten())
    net.add(gluon.nn.Dense(128, activation="relu"))
    net.add(gluon.nn.Dense(10))

net.initialize(ctx=ctx)

In [2]:
batch_size = 256
train_data, test_data = utils.load_data_fashion_mnist(batch_size)
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.5})

for epoch in range(5):
    train_loss = 0.
    train_acc = 0.
    for data, label in train_data:
        label = label.as_in_context(ctx)
        with autograd.record():
            output = net(data.as_in_context(ctx))
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        trainer.step(batch_size)

        train_loss += nd.mean(loss).asscalar()
        train_acc += utils.accuracy(output, label)

    test_acc = utils.evaluate_accuracy(test_data, net, ctx)
    print("Epoch %d. Loss: %f, Train acc %f, Test acc %f" % (
            epoch, train_loss/len(train_data), train_acc/len(train_data), test_acc))

Epoch 0. Loss: 1.085110, Train acc 0.589284, Test acc 0.778125
Epoch 1. Loss: 0.529853, Train acc 0.792448, Test acc 0.833203
Epoch 2. Loss: 0.452444, Train acc 0.827261, Test acc 0.844531
Epoch 3. Loss: 0.414742, Train acc 0.843999, Test acc 0.856152
Epoch 4. Loss: 0.385881, Train acc 0.853175, Test acc 0.863867
